In [ ]:
%load_ext autoreload

In [ ]:
%autoreload 2

In [ ]:
!pip install tqdm --upgrade

In [ ]:
!pip install lorem

In [ ]:
!pip install google_images_download

In [ ]:
from openpyxl import load_workbook
from tqdm.notebook import tqdm
from copy import deepcopy
from arrow import Arrow
from datetime import datetime, timedelta
from random import choice
from django_bulk_update.helper import bulk_update
from IPython.display import display, Markdown
import lorem
import json
import os
import random
import copy
import math

In [ ]:
from fixtures.mock_empreiteira.permission_names import pretty_print_permission
from helpers.validators.brazilian_documents import generate_cnpj
from helpers.strings import get_autonumber_array, clean_latin_string
from helpers.apps.services import (
    create_or_update_services_and_usages,
    create_services_from_measurement,
)

# EMPREITEIRA

In [ ]:
company_name = "Demonstração Empreiteiras"
prefixo_folder = "fixtures/mock_empreiteira/"
prefixo_users = "demo."
admin_user_count = 4
campo_user_count = 25

In [ ]:
companies = Company.objects.filter(name=company_name)

In [ ]:
company = companies.first()

## WARNING! The following two cells will delete an existing company with the same name

In [ ]:
User.objects.filter(companies__in=companies).delete()

In [ ]:
companies.delete()

## UserPermission

Create two permission levels

In [ ]:
with open(prefixo_folder + 'admin_permissions.json', 'r+') as fo:
    admin_json = json.load(fo)

In [ ]:
pretty_print_permission(admin_json)

In [ ]:
with open(prefixo_folder + 'campo_permissions.json', 'r+') as fo:
    campo_json = json.load(fo)

In [ ]:
pretty_print_permission(campo_json)

In [ ]:
admin_perm = UserPermission.objects.create(
    name="Administrador",
    permissions=admin_json
)
campo_perm = UserPermission.objects.create(
    name="Campo",
    permissions=campo_json
)

## User

Create three users, data can be random. If possible, have a list of first_names and last_names and generate a random combination.

In [ ]:
with open(prefixo_folder + 'first_names.json', 'r+') as fo:
    first_names = json.load(fo)
    
with open(prefixo_folder + 'last_names.json', 'r+') as fo:
    last_names = json.load(fo)
    
def get_user_params():
    first = choice(first_names)
    last = choice(last_names)
    username = prefixo_users + first.lower() + "." + last.lower()
    return first, last, username

In [ ]:
admin_users = []
for _ in range(admin_user_count):
    first, last, username = get_user_params()
    while User.objects.filter(username=username):
        first, last, username = get_user_params()
    admin = User.objects.create(
        first_name=first,
        last_name=last,
        username=username,
    )
    admin.set_password('demo.admin')
    admin.save()
    admin_users.append(admin)
    
campo_users = []
for _ in range(campo_user_count):
    first, last, username = get_user_params()
    while User.objects.filter(username=username):
        first, last, username = get_user_params()
    campo = User.objects.create(
        first_name=first,
        last_name=last,
        username=username
    )
    campo.set_password('demo.campo')
    campo.save()
    campo_users.append(campo)

In [ ]:
# Print username and password

display(Markdown("Admin users - password is demo.admin"))
for user in admin_users:
    print(user.username)
    
display(Markdown("Campo users - password is demo.campo"))
for user in campo_users:
    print(user.username)

## Company

Create a Company. The name will be provided when the script runs.

In [ ]:
with open(prefixo_folder + 'metadata.json', 'r+') as fo:
    metadata = json.load(fo)

In [ ]:
with open(prefixo_folder + 'custom_options.json', 'r+') as fo:
    custom_options = json.load(fo)

In [ ]:
company = Company.objects.create(
    name=company_name,
    metadata=metadata,
    custom_options=custom_options,
    active=True,
    cnpj=generate_cnpj(),
    owner=admin_users[0]
)

In [ ]:
# Add company in User Permissions

admin_perm.companies.add(company)
admin_perm.save()
campo_perm.companies.add(company)
campo_perm.save()

## UserInCompany

Create three objects, tying the three users to the company, one with each UserPermission

In [ ]:
for user in admin_users:
    user_in_company_admin = UserInCompany.objects.create(
        user=user,
        company=company,
        permissions=admin_perm
    )
    
for user in campo_users:
    user_in_company_campo = UserInCompany.objects.create(
        user=user,
        company=company,
        permissions=campo_perm
    )

## CompanyGroup

Create a CompanyGroup with the same name as the company.

Set the CompanyGroup of the Company and all the users to be the newly created one.

In [ ]:
company_group = CompanyGroup.objects.create(
    name=company_name,
    key_user=admin_users[0]
)

In [ ]:
company.company_group = company_group
company.save()

for user in admin_users + campo_users:
    user.company_group = company_group
    user.save()

## Firm

Create one object for admins and several for the other users

In [ ]:
with open(prefixo_folder + 'firm_names.json', 'r+') as fo:
    firm_names = json.load(fo)

In [ ]:
Firm.objects.filter(company=company).delete()

In [ ]:
firm_admins = Firm.objects.create(
    name="Administração",
    manager=admin,
    company=company,
    cnpj=generate_cnpj(),
    is_company_team=True,
    members_amount=5
)
firms_campo = []
for i in range(8):
    name = choice(firm_names)
    while Firm.objects.filter(company=company, name=name):
        name = choice(firm_names)
        
    firm_apontadores = Firm.objects.create(
        name=name,
        manager=campo_users[i],
        company=company,
        cnpj=generate_cnpj(),
        is_company_team= True,
        members_amount = choice([3, 4, 5, 6, 7])
    )
    firms_campo.append(firm_apontadores)

## UserInFirm

Add the users to the corresponding firms

In [ ]:
for user in admin_users:
    user_in_firm_admin = UserInFirm.objects.create(
        user=user,
        firm=firm_admins
    )
    
for user in campo_users:
    if user.user_firms_manager.first():
        firm = user.user_firms_manager.first()
    else:
        firm = choice(firms_campo)
    user_in_firm_campo = UserInFirm.objects.create(
        user=user,
        firm=firm
    )

# Equipment


Create a few Equipment objects.

In [ ]:
Equipment.objects.filter(company=company).delete()

In [ ]:
for i in range(10):
    letter = chr(ord('A')+i)
    plate = letter * 3 + str(i) * 4
    equipment = Equipment(
        plate = plate,
        description = choice(["Carro", "Camioneta", "Caminhão"]),
        manager = choice(firms_campo).manager,
        company = company,
        is_rented = False,
        active = choice([True, True, False])
    )
    equipment.save()

## History

Some RDO dashboard plots work based on the history of the objects. Therefore, create random changes in the history of Firm and Equipment for the past year

In [ ]:
for user in User.objects.filter(companies=company):
    hist = user.history.earliest()
    hist.history_date = hist.history_date - timedelta(days=365)
    hist.save()
    user.history.exclude(history_id=hist.history_id).delete()

In [ ]:
for firm in Firm.objects.filter(company=company):
    hist = firm.history.earliest()
    hist.history_date = hist.history_date - timedelta(days=365)
    hist.save()
    firm.history.exclude(history_id=hist.history_id).delete()

In [ ]:
for uif in UserInFirm.objects.filter(user__companies=company):
    hist = uif.history.earliest()
    hist.history_date = hist.history_date - timedelta(days=365)
    hist.save()
    uif.history.exclude(history_id=hist.history_id).delete()

In [ ]:
for firm in Firm.objects.filter(company=company):
    first_hist = firm.history.earliest()
    hist_dates = Arrow.range("day", first_hist.history_date, timezone.now())
    
    history_type = "~"
    history_user = admin_users[0]

    historical_instances = []
    instance = firm
    hist_model = first_hist.instance.history.model
    for hist_date in hist_dates:
        row = hist_model(
            history_date=hist_date.datetime,
            history_user=history_user,
            history_change_reason="",
            history_type=history_type,
            **{
                field.attname: getattr(instance, field.attname)
                for field in instance._meta.fields
                if (field.name not in hist_model._history_excluded_fields) and (field.name not in ['active', 'members_amount'])
            },
            active= choice([True, True, False]),
            members_amount = choice([3, 4, 5, 6, 7])
            
        )
        if hasattr(instance, "history_relation"):
            row.history_relation_id = instance.pk
        historical_instances.append(row)

    hist_model.objects.bulk_create(
        historical_instances, batch_size=100
    )

In [ ]:
for equipment in Equipment.objects.filter(company=company):
    hist = equipment.history.earliest()
    hist.history_date = hist.history_date - timedelta(days=365)
    hist.save()
    equipment.history.exclude(history_id=hist.history_id).delete()

In [ ]:
for equipment in Equipment.objects.filter(company=company):
    first_hist = equipment.history.earliest()
    hist_dates = Arrow.range("day", first_hist.history_date, timezone.now())
    
    history_type = "~"
    history_user = admin_users[0]

    historical_instances = []
    instance = equipment
    hist_model = first_hist.instance.history.model
    for hist_date in hist_dates:
        row = hist_model(
            history_date=hist_date.datetime,
            history_user=history_user,
            history_change_reason="",
            history_type=history_type,
            **{
                field.attname: getattr(instance, field.attname)
                for field in instance._meta.fields
                if (field.name not in hist_model._history_excluded_fields) and (field.name not in ['active', 'members_amount'])
            },
            active=choice([True, True, False]),
            
        )
        if hasattr(instance, "history_relation"):
            row.history_relation_id = instance.pk
        historical_instances.append(row)

    hist_model.objects.bulk_create(
        historical_instances, batch_size=100
    )

## TileLayers

Create using this objects below and tie all of them to the newly created company.

In [ ]:
tile_objects = [{"type": "mapbox", "description": "Satélite", "name": "Satélite", "provider_info": {"url": "https://api.mapbox.com/styles/v1/natank/cjmmhqxpt0jsn2smsihy5nx5w/tiles/{z}/{x}/{y}.mapbox", "type": "mapbox", "order": 3, "accessToken": "pk.eyJ1IjoibmF0YW5rIiwiYSI6ImNqbW1oNngyYjBoZWEzcm83dDRnZTRmMnoifQ.IJHj30XcphWWzx7Luvc-2A", "attribution": "Licensed by &copy; <a href=\"https://www.mapbox.com/\">Mapbox</a>", "styleString": "mapbox://styles/natank/ck0z9h5s014wj1cqum4jjssbk"}},
{"type": "mapbox", "description": "Mapa Claro", "name": "Mapa Claro", "provider_info": {"url": "https://api.mapbox.com/styles/v1/natank/cjmmhqxpt0jsn2smsihy5nx5w/tiles/{z}/{x}/{y}.mapbox", "type": "mapbox", "order": 4, "accessToken": "pk.eyJ1IjoibmF0YW5rIiwiYSI6ImNqbW1oNngyYjBoZWEzcm83dDRnZTRmMnoifQ.IJHj30XcphWWzx7Luvc-2A", "attribution": "Licensed by &copy; <a href=\"https://www.mapbox.com/\">Mapbox</a>", "styleString": "mapbox://styles/natank/ck0z9bp0r0z4l1cqhp4wt15ew"}},
{"type": "mapbox", "description": "Mapa Escuro", "name": "Mapa Escuro", "provider_info": {"url": "https://api.mapbox.com/styles/v1/natank/cjmmhqxpt0jsn2smsihy5nx5w/tiles/{z}/{x}/{y}.mapbox", "type": "mapbox", "order": 5, "accessToken": "pk.eyJ1IjoibmF0YW5rIiwiYSI6ImNqbW1oNngyYjBoZWEzcm83dDRnZTRmMnoifQ.IJHj30XcphWWzx7Luvc-2A", "attribution": "Licensed by &copy; <a href=\"https://www.mapbox.com/\">Mapbox</{a>", "styleString": "mapbox://styles/natank/ck0z9os2914au1cqqaj464p5m"}},
{"type": "openStreetMap", "description": "OpenStreetMap", "name": "OpenStreetMap", "provider_info": {"url": "https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png", "type": "openStreetMap", "order": 6, "attribution": "&amp;copy <a href=\"http://osm.org/copyright\">OpenStreetMap</a> contributors"}},
{"type": "mapbox", "description": "Relevo", "name": "Relevo", "provider_info": {"url": "https://api.mapbox.com/styles/v1/natank/cjmmhqxpt0jsn2smsihy5nx5w/tiles/{z}/{x}/{y}.mapbox", "type": "mapbox", "order": 2, "accessToken": "pk.eyJ1IjoibmF0YW5rIiwiYSI6ImNqbW1oNngyYjBoZWEzcm83dDRnZTRmMnoifQ.IJHj30XcphWWzx7Luvc-2A", "attribution": "Licensed by &copy; <a href=\"https://www.mapbox.com/\">Mapbox</a>", "styleString": "mapbox://styles/natank/ck0z7y0bi0g6x1cq1oxwo8bhd"}},
{"type": "mapbox", "description": "Padrão", "name": "Padrão", "provider_info": {"url": "https://api.mapbox.com/styles/v1/natank/cjmmhqxpt0jsn2smsihy5nx5w/tiles/{z}/{x}/{y}.mapbox", "type": "mapbox", "order": 1, "accessToken": "pk.eyJ1IjoibmF0YW5rIiwiYSI6ImNqbW1oNngyYjBoZWEzcm83dDRnZTRmMnoifQ.IJHj30XcphWWzx7Luvc-2A", "attribution": "Licensed by &copy; <a href=\"https://www.mapbox.com/\">Mapbox</a>", "styleString": "mapbox://styles/natank/cjn50jrc82oyq2rlew8h2c7yp"}}]

In [ ]:
for item in tile_objects:
    tile_layer = TileLayer.objects.create(**item)
    tile_layer.companies.add(company)
    tile_layer.save()

## ShapeFile

not necessary for now

## OccurrenceType and OccurrenceTypeSpecs

Create objects tying the OccurrenceType objects to the new Company. Randomize the color.

In [ ]:
new_forms = []

forms_folder = prefixo_folder + "formularios"
for index, file in enumerate(os.listdir(forms_folder)):
    if '.xlsx' in file and not "~" in file:
        excel_file = file
    if not '.txt' in file:
        continue
    file_name = '{}/{}'.format(forms_folder, file)
    print('------')
    print(file_name)
    with open(file_name, 'r+') as fo:
        form = json.load(fo)
        print(form['displayName'])
        new_forms.append(form)

In [ ]:
wb = load_workbook('{}/{}'.format(forms_folder, excel_file))
sheetname = wb.sheetnames[2]
ws = wb[sheetname]

In [ ]:
header = []
values = []

for index, row in enumerate(ws.rows):
    if index == 0:
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
    values.append(obj)

In [ ]:
for form in new_forms:
    if form['displayName'] not in [a['Classe'] for a in values]:
        print(form['displayName'])
    else:
        row = next(a for a in values if a['Classe'] == form['displayName'])
        form['kind'] = row['id_Natureza']
        if row['Prazo']:
            if 'h' in row['Prazo']:
                form['deadline'] = timedelta(hours=int(row['Prazo'].split(' ')[0]))
            elif 'dias' in row['Prazo']:
                form['deadline'] = timedelta(days=int(row['Prazo'].split(' ')[0]))
            else:
                print('DEU RUIM')

In [ ]:
diario_fields = {"id": 1, "name": "dailyReport", "fields": [{"id": 1, "api_name": "weather", "data_type": "select", "display_name": "Clima", "select_options": {"options": [{"name": "Bom", "value": "1"}, {"name": "Inst\u00e1vel", "value": "2"}, {"name": "Chuvoso", "value": "3"}, {"name": "Impratic\u00e1vel", "value": "4"}]}}, {"id": 2, "api_name": "notes", "data_type": "textArea", "display_name": "Observa\u00e7\u00e3o"}, {"id": 3, "api_name": "rain", "data_type": "float", "display_name": "Pluviometria"}, {"id": 4, "api_name": "colaborators", "data_type": "textArea", "display_name": "Colaboradores presentes"}, {"id": 5, "api_name": "signDuration", "data_type": "float", "display_name": "Tempo de Coloca\u00e7\u00e3o da Sinaliza\u00e7\u00e3o (min)"}, {"id": 6, "api_name": "signDuration2", "data_type": "float", "display_name": "Tempo de Retirada da Sinaliza\u00e7\u00e3o (min)"}], "groups": [{"order": 1, "members": [1, 3], "display_name": "Clima"}, {"order": 2, "members": [2, 4, 5, 6], "display_name": "Informa\u00e7\u00f5es Adicionais"}], "display_name": "Di\u00e1rio", "measurementColumns": [{"key": "weather", "logic": {"if": [{"==": [{"var": "formData.weather"}, "1"]}, "Bom", {"==": [{"var": "formData.weather"}, "2"]}, "Inst\u00e1vel", {"==": [{"var": "formData.type"}, "3"]}, "Chuvoso", {"==": [{"var": "formData.type"}, "4"]}, "Impratic\u00e1vel", 0]}, "width": 18, "header": "Clima"}, {"key": "notes", "logic": {"var": "formData.notes"}, "width": 20, "header": "Observa\u00e7\u00f5es"}, {"key": "rain", "logic": {"var": "formData.rain"}, "width": 16, "header": "Pluviometria"}, {"key": "colaborators", "logic": {"var": "formData.colaborators"}, "width": 16, "header": "Colaboradores na frente"}, {"key": "signDuration", "logic": {"var": "formData.signDuration"}, "width": 16, "header": "Tempo de Coloca\u00e7\u00e3o da Sinaliza\u00e7\u00e3o (min)"}, {"key": "signDuration2", "logic": {"var": "formData.signDuration2"}, "width": 16, "header": "Tempo de Retirada da Sinaliza\u00e7\u00e3o (min)"}]}
diario_fields['displayName'] = "Diário"
diario_fields['kind'] = 6
new_forms.append(diario_fields)

In [ ]:
occurrence_types = []
for form in new_forms:
    form_copy = deepcopy(form)
    if 'deadline' in form_copy.keys():
        del form_copy['deadline']
    if 'kind' in form_copy.keys():
        del form_copy['kind']
    occurrence_type = OccurrenceType(
        name=form['displayName'],
        occurrence_kind=form['kind'],
        form_fields = form_copy,
        created_by=User.objects.get(username='rlcs'),
        deadline= form['deadline'] if 'deadline' in form else None
    )
    occurrence_types.append(occurrence_type)

In [ ]:
for occurrence_type in occurrence_types:
    occurrence_type.save()
    spec = OccurrenceTypeSpecs(
        occurrence_type = occurrence_type,
        company = company,
        color = "#"+''.join([random.choice('0123456789ABCDEF') for j in range(6)])
    )
    spec.save()

In [ ]:
def get_color():
    return "#%06x" % random.randint(0, 0xFFFFFF)

## ServiceOrderActionStatus and ServiceOrderActionStatusSpecs

Create copies of the ones from Arteris Litoral Sul

Create one for each status

In [ ]:
status_identificado = ServiceOrderActionStatus.objects.create(
    name="Identificado",
    kind="REPORTING_STATUS"
)
status_programado = ServiceOrderActionStatus.objects.create(
    name="Programado",
    kind="REPORTING_STATUS"
)
status_executado = ServiceOrderActionStatus.objects.create(
    name="Executado",
    kind="REPORTING_STATUS"
)

status_specs_identificado = ServiceOrderActionStatusSpecs.objects.create(
    company=company,
    status=status_identificado,
    color=get_color(),
    order=1
)
status_specs_programado = ServiceOrderActionStatusSpecs.objects.create(
    company=company,
    status=status_programado,
    color=get_color(),
    order=2
)
status_specs_executado = ServiceOrderActionStatusSpecs.objects.create(
    company=company,
    status=status_executado,
    color=get_color(),
    order=3
)

## Road

Create a single demo road containing a part of BR282-SC

In [ ]:
road = Road(
    marks = {"0": {"km": 114, "key": 0, "index": 0, "point": {"type": "Point", "coordinates": [-49.3880981, -27.7483643]}}, "1": {"km": 223, "key": 1, "index": 745, "point": {"type": "Point", "coordinates": [-50.363131, -27.800003]}}, "2": {"km": 223.001, "key": 2, "index": 745, "point": {"type": "Point", "coordinates": [-50.363131, -27.800003]}}},
    direction = 0,
    name = "BR282-SC-DEMO",
    uf = "42",
)
road.save()
road.company.add(company)

## Service and ServiceSpecs

We'll create these based on an excel sheet. I'm attaching an example sheet, and also a notebook with some code we usually use to import the file.

In [ ]:
import time
import json
import codecs
from datetime import datetime, timezone
from openpyxl import load_workbook
from difflib import SequenceMatcher
from operator import __or__ as OR
from functools import reduce
from helpers.histories import bulk_update_with_history
from helpers.apps.services import create_usages_from_reporting, create_or_update_services_and_usages
import copy

In [ ]:
from django.core.exceptions import MultipleObjectsReturned, ObjectDoesNotExist
from django.db.models import Q

In [ ]:
from simple_history.utils import bulk_create_with_history
from helpers.apps.json_logic import apply_reporting_json_logic
from apps.services.serializers import ServiceSerializer, ServiceSpecsSerializer

In [ ]:
def fix_field(trimmed, fields):

    value = trimmed
    
    # FIX ONES WITH data. IN FRONT
    if "data.formData" in value:
        value = value.replace(
            "data.formData", "formData"
        )
        
    # FIX ONES WITHOUT formData. IN FRONT
    if (
        not "formData" in value
        and "formData." + value in fields
    ):
        value = value.replace(
            value, "formData." + value
        )
        
    # FIX ONES WITH WRONT height heigth
    if value.replace("ht", "th") in fields:
        value = value.replace("ht", "th")

    # FIX ONES WITH WRONT heigth height
    if value.replace("th", "ht") in fields:
        value = value.replace("th", "ht")
        
    return value

In [ ]:
def get_nearest(field, fields):
    ratios = [SequenceMatcher(None, field, a).ratio() for a in fields]
    argmax = -max((x,-i) for i,x in enumerate(ratios))[1]
    return fields[argmax]

In [ ]:
def get_service(all_services, name, kind):
    filtered = [a for a in all_services if a.name == name and str(a.kind) == str(kind)]
    if len(filtered) < 1:
        print("---- ERRO")
        print('Nenhum serviço encontrado: {} - {}'.format(kind, name))
        return "none"
    if len(filtered) > 1:
        print("---- ERRO")
        print('Múltiplos serviços encontrados: {} - {}'.format(kind, name))
        return "multiple"
    else:
        return next(
            a for a in all_services if a.name == name and str(a.kind) == str(kind)
        )

In [ ]:
def get_type(all_types, name):
    filtered = [a for a in all_types if a.name == name]
    if len(filtered) != 1:
        return None
    else:
        return next(
            a for a in all_types if a.name == name
        )

In [ ]:
def get_spec(all_specs, service, occurrence_type):
    filtered = [a for a in all_specs if a.service.uuid == service.uuid and a.occurrence_type.uuid == occurrence_type.uuid]
    if len(filtered) != 1:
        return None
    else:
        return next(
            a for a in all_specs if a.service.uuid == service.uuid and a.occurrence_type.uuid == occurrence_type.uuid
        )

In [ ]:
filename = prefixo_folder + "Contrato"
wb = load_workbook(filename + ".xlsx")
sheetname = wb.sheetnames[0]
ws = wb[sheetname]

In [ ]:
header = []
values = []
good_fields = []
bad_fields = []

all_types = OccurrenceType.objects.filter(company__in=[company])

for index, row in enumerate(ws.rows):
    if index == 0 or not header:
        if row[0].value == None:
            continue
        header = list([a.value for a in row])
        continue
    obj = {}
    for col_index, cell in enumerate(row):
        value = cell.value
        obj[header[col_index]] = value
        if header[col_index] and "Formula" in header[col_index] and value is not None:
            formula_index = header[col_index].strip("Formula")
            type_name = obj["Type" + formula_index]
            if type_name is None:
                continue
                
            oType = get_type(all_types, type_name)
            if not oType:
                print("---- ERRO")
                print("Formulário não encontrado: {}".format(type_name))
                continue
                    
            else:
                try:
                    fields = [
                        "formData." + a["apiName"]
                        for a in oType.form_fields["fields"]
                    ]
                    touched = False
                    temp_value = value
                    for var in value.split('"var":')[1:]:
                        trimmed = var.split('"}')[0].strip().strip('"')
                        if trimmed == "km":
                            good_fields.append(trimmed)
                            continue
                        if not trimmed in fields:
                            bad_field = {
                                "service": obj['name'],
                                "occurrence_type": oType,
                                "field": trimmed,
                                "nearest": get_nearest(trimmed,fields),
                                "cell": cell,
                                "fixed": False,
                                "fixed_value": ""
                            }
                            fixed_field = fix_field(trimmed, fields)
                            if fixed_field != trimmed:
                                temp_value = temp_value.replace(trimmed, fixed_field)
                                touched = True
                                bad_field["fixed"] = True
                                bad_field["fixed_value"] = fixed_field
                            bad_fields.append(bad_field)
                        else:
                            good_fields.append(trimmed)
                            
                    if touched:
                        cell.value = temp_value
                except Exception as e:
                    print(e)
                    pass
            try:
                obj[header[col_index]] = json.loads(temp_value)
            except Exception as e:
                print(str(e))
                print(value)
    values.append(obj)

In [ ]:
print("good: ", len(good_fields))
print("fixed: ", len([a for a in bad_fields if a['fixed']]))
print("bad: ", len([a for a in bad_fields if not a['fixed']]))
print("----")
for field in bad_fields:
    print("CÉLULA {}, SERVIÇO '{}', FORMULARIO '{}'".format(field["cell"], field["service"], field["occurrence_type"].name))
    if field['fixed']:
        print("CAMPO '{}' corrigido automaticamente para {}".format(field['field'], field['fixed_value']))
    else:
        print("CAMPO '{}' não existe".format(field['field']))
        print("Correspondência mais próxima: '{}'".format(field['nearest']))
    print("----")

In [ ]:
services = values

In [ ]:
for service in services:
    all_types = [b for a,b in service.items() if "Type " in a and b]
    for index, otype in enumerate(all_types):
        if index != all_types.index(otype):
            print("DUPLICATED -  service {} and type {} - ignoring".format(service['name'], service['Type ' + str(index + 1)]))
            service['Type ' + str(index + 1)] = ""

In [ ]:
def parse_numbers(number):
    try:
        if isinstance(number, (int, float)):
            return number
        elif "-" in number:
            return 0
        else:
            return round(float(number.replace(".", "").replace(",", ".")), 3)
    except:
        print("ERROR PARSING NUMBER!!")
        return 0

In [ ]:
def create_service(service):
    serializer = ServiceSerializer()
    new_obj = serializer.create(
        validated_data={
            "name": service["name"],
            "group": service["group"] or "",
            "company": company,
            "kind": service["kind"],
            "code": service["code"],
            "unit": service["unit"],
            "unit_price": parse_numbers(service["unit_price"]),
            "adjustment_coefficient": parse_numbers(
                service["adjustment_coefficient"] or 1
            ),
            "total_amount": 0,
            "current_balance": parse_numbers(service["current_balance"]),
        }
    )
    print("---- INFO")
    print("Criando serviço {}".format(service['name']))
    return new_obj

In [ ]:
def update_service(existing_service, service):
    needs_update = False
    updated_fields = []
    new_values = {}
    
    new_values["group"] = service["group"] or ""
    new_values["code"] = service["code"]
    new_values["unit"] = service["unit"]
    new_values["unit_price"] = parse_numbers(service["unit_price"])
    new_values["adjustment_coefficient"] = parse_numbers(
        service["adjustment_coefficient"] if 'adjustment_coefficient' in service else 1
    )
    new_values["total_amount"] = 0
    new_values["current_balance"] = parse_numbers(
        service["current_balance"]
    )
    for field, value in new_values.items():
        if getattr(existing_service, field) != value:
            updated_fields.append((field, getattr(existing_service, field), value))
            needs_update = True
            
    if needs_update:
        existing_service.group = service["group"] or ""
        existing_service.code = service["code"]
        existing_service.unit = service["unit"]
        existing_service.unit_price = parse_numbers(service["unit_price"])
        existing_service.adjustment_coefficient = parse_numbers(
            service["adjustment_coefficient"] if 'adjustment_coefficient' in service else 1
        )
        existing_service.total_amount = 0
        existing_service.current_balance = parse_numbers(
            service["current_balance"]
        )
        existing_service.save()

        for field, old_value, new_value in updated_fields:
            print("---- INFO")
            print("{} - ATUALIZADO {} de {} para {}".format(existing_service.name, field, old_value, new_value))

In [ ]:
def create_spec(service, occurrence_type, formula):
    print("---- INFO")
    print("Criando formula para {} e {}".format(service.name, occurrence_type.name))
    serializer = ServiceSpecsSerializer()
    new_obj = serializer.create(
        validated_data={
            "service": service,
            "occurrence_type": occurrence_type,
            "formula": formula,
        }
    )
    return new_obj

In [ ]:
def update_spec(existing_spec, formula):
    if existing_spec.formula != formula:
        existing_spec.formula = formula
        existing_spec.save()
        print("---- INFO")
        print("Formula entre {} e {} foi atualizada".format(existing_spec.service.name, existing_spec.occurrence_type.name))
        return existing_spec

In [ ]:
all_services = Service.objects.filter(company=company)
all_types = OccurrenceType.objects.filter(company__in=[company])
all_specs = ServiceSpecs.objects.filter(service__in=all_services, occurrence_type__in=all_types)\
    .select_related('service', 'occurrence_type')
updated_specs = []

for service in tqdm(services):
    existing_service = get_service(all_services, service['name'], service['kind'])
    if existing_service == "none":
        existing_service = create_service(service)
    elif existing_service == "multiple":
        continue
    else:
        update_service(existing_service, service)        
        
    type_num = len([a for a in service.keys() if "Type " in a])
    
    for i in range(1, type_num + 1):
        if "Type " + str(i) in service.keys():
            try:
                typeName = service["Type " + str(i)]
                if typeName is None:
                    continue
                
                oType = get_type(all_types, typeName)
                if not oType:
                    print("---- ERRO")
                    print("SERVIÇO: {}, COLUNA {}, FORMULÁRIO {} NÃO ENCONTRADO".format(existing_service.name, i, typeName))
                    continue
                    
                formula = service["Formula " + str(i)]

                existing_spec = get_spec(all_specs, existing_service, oType)
                if existing_spec:
                    updated = update_spec(existing_spec, formula)
                else:
                    updated = create_spec(existing_service, oType, formula)
                
                if updated:
                    updated_specs.append(updated)
                
            except Exception as e:
                print(typeName, str(e))

## Reporting

Generate 10000 new objects according to the instructions below:

```
number: autogenerate
company: newly created company
road_name: pick a random one
road: same as above
km: random within the range of the selected road
point: let the algorithm calculate
direction: pick a random one according to whats available in the company custom_options
lane: pick a random one according to whats available in the company custom_options
created_by: one of the newly created users
firm: the firm of the user
occurrence_type: pick a random one
form_data: randomly fill according to the form_fields of the selected occurrence_type
executed_at: random between None and a date in the past year
created_at: a random date in the past year. If executed_at was filled, make sure this date is before executed_at
found_at: a random date in the past year. If executed_at was filled, make sure this date is before executed_at
updated_at: a random date in the past year. If executed_at was filled, make sure this date is before executed_at
status: pick a random one
```

In [ ]:
from datetime import date, timedelta, datetime
from django.utils import timezone
# from pytz import timezone as tz

def get_dates(firm, user):
    job = None
    executed_at = None
    
    today_date = timezone.now()
    start_date = today_date.toordinal()
    end_date = today_date.replace(year=today_date.year-1).toordinal()
    rand = date.fromordinal(random.randint(end_date, start_date))
    dt = datetime.combine(rand, datetime.min.time())
    created_at = (dt + timedelta(hours=5)).replace(tzinfo=today_date.tzinfo)
    found_at = (created_at - timedelta(hours=random.randint(0, 23))).replace(hour = random.randint(7, 19))
    updated_at = (created_at + timedelta(days=random.randint(0, 7))).replace(hour = random.randint(7, 19))
    executed = (created_at + timedelta(days=random.randint(0, 7))).replace(hour = random.randint(7, 19))
    
    jobs = Job.objects.filter(
        company=company,
        start_date__lte=found_at,
        end_date__gte=found_at,
    )
    if not jobs:
        jobs = [
            Job.objects.create(
                company=company,
                start_date=found_at + timedelta(days=2),
                end_date=found_at + timedelta(days=20),
                firm=firm,
                created_by=user,
                title=lorem.sentence(),
                worker=user
            )
        ]

    if today_date - found_at > timedelta(days=30):
        status = status_executado
    else:
        status = choice(ServiceOrderActionStatus.objects.filter(companies=company))
    
    if status == status_executado:
        executed_at = executed
        job = choice(jobs)
    elif status == status_programado:
        job = choice(jobs)
        
    
    return created_at, found_at, updated_at, executed_at, job, status

In [ ]:
import re

def snake_case(name):
    s1 = re.sub("(.)([A-Z][a-z]+)", r"\1_\2", name)
    return re.sub("([a-z0-9])([A-Z])", r"\1_\2", s1).lower()

def snake_dict(in_dict):
    out_dict = {}
    for key, value in in_dict.items():
        out_dict[snake_case(key)] = value
    return out_dict

In [ ]:
def get_form_data(occ_type):
    form_data = {}
    if occ_type.form_fields:
        if "fields" in occ_type.form_fields:
            fields = occ_type.form_fields["fields"]
            for in_field in fields:
                field = snake_dict(in_field)
                if "api_name" in field and "data_type" in field:
                    if field["data_type"] == "float":
                        if field["api_name"] == "width":
                            form_data[field["api_name"]] = random.choice([3.8, 7.6])
                        elif field["api_name"] == "length":
                            if occ_type.occurrence_kind == "1":
                                form_data[field["api_name"]] = round(random.uniform(1, 5), 2)
                            else:
                                form_data[field["api_name"]] = round(random.uniform(5, 50), 2)
                        elif field["api_name"] == "height":
                            form_data[field["api_name"]] = round(random.uniform(0.01, 0.2), 2)
                        elif field["api_name"] == "rain":
                            form_data[field["api_name"]] = round(random.random() * 10, 2)
                        else:
                            form_data[field["api_name"]] = round(random.uniform(0, 1), 2)
                    elif field["data_type"] == "string":
                        form_data[field["api_name"]] = " ".join(lorem.sentence().split(" ")[0:2])
                    elif field["data_type"] == "number":
                        form_data[field["api_name"]] = int(random.uniform(0, 1)*10)
                    elif field["data_type"] == "textArea":
                        form_data[field["api_name"]] = lorem.sentence()
                    elif field["data_type"] == "boolean":
                        form_data[field["api_name"]] = True
                    elif field["data_type"] == "select":
                        if "select_options" in field:
                            if "options" in field["select_options"]:
                                value = field["select_options"]["options"][0]["value"]
                                form_data[field["api_name"]] = value
                    elif field["data_type"] == "selectMultiple":
                        if "select_options" in field:
                            if "options" in field["select_options"]:
                                value = field["select_options"]["options"][0]["value"]
                                form_data[field["api_name"]] = [value]

    return form_data

In [ ]:
direction_options = custom_options["reporting"]["fields"]["direction"]["selectOptions"]["options"]
directions = [item["value"] for item in direction_options]

lane_options = custom_options["reporting"]["fields"]["lane"]["selectOptions"]["options"]
lanes = [item["value"] for item in lane_options]

In [ ]:
Reporting.objects.filter(company=company).delete()

In [ ]:
Job.objects.filter(company=company).delete()

In [ ]:
all_types = list(OccurrenceType.objects.filter(company=company).exclude(occurrence_kind__in=["2", "6"]))
random.shuffle(all_types)
# internal_types = all_types[:math.ceil(len(all_types)/2)]
# external_types = all_types[math.ceil(len(all_types)/2):]

In [ ]:
roads = Road.objects.filter(company=company)

Create inventory

In [ ]:
inventory_types = OccurrenceType.objects.filter(occurrence_kind="2", company=company)

for i in tqdm(range(100)):
    road = choice(roads)
    kms = [road.marks[item]["km"] for item in road.marks]
    occ_type = choice(inventory_types)
    
    km = round(random.uniform(min(kms), max(kms)), 2)
    
    Reporting.objects.create(
        company=company,
        road_name=road.name,
        road=road,
        km=km,
        end_km=km,
        direction=choice(directions),
        lane=choice(lanes[:4]),
        occurrence_type=occ_type,
        form_data=get_form_data(occ_type),
        created_at=timezone.now().replace(year=2019),
        updated_at=timezone.now().replace(year=2019),
        found_at=timezone.now().replace(year=2019),
    )

Create reportings associated to the inventory

In [ ]:
for inventory in tqdm(Reporting.objects.filter(occurrence_type__occurrence_kind="2", company=company)):
    for i in range(choice([2, 3, 4])):
        firm = choice(firms_campo)
        occ_type = choice(OccurrenceType.objects.filter(
            company=company, 
            name__icontains=inventory.occurrence_type.name
        ).exclude(occurrence_kind="2"))
        created, found, updated, executed, job, status = get_dates(firm, firm.manager)

        Reporting.objects.create(
            company=company,
            road_name=inventory.road.name,
            road=inventory.road,
            km=inventory.km,
            end_km=inventory.km + (inventory.form_data['length']/1000),
            direction=inventory.direction,
            lane=inventory.lane,
            firm=firm,
            created_by=firm.manager,
            occurrence_type=occ_type,
            form_data=inventory.form_data,
            status=status,
            executed_at=executed,
            created_at=created,
            updated_at=updated,
            found_at=found,
            job=job,
            parent=inventory
        )

Create random reportings of all other types

In [ ]:
for i in tqdm(range(500)):
    road = choice(roads)
    kms = [road.marks[item]["km"] for item in road.marks]
#     firm = choice(Firm.objects.filter(company=company))
    firm = choice(firms_campo)
#     if firm.is_company_team:
#         occ_type = choice(internal_types)
#     else:
#         occ_type = choice(external_types)
    occ_type = choice(all_types)
#     occ_type = choice(OccurrenceType.objects.filter(company=company).exclude(occurrence_kind="6"))
    created, found, updated, executed, job, status = get_dates(firm, firm.manager)
    
    Reporting.objects.create(
        company=company,
        road_name=road.name,
        road=road,
        km=round(random.uniform(min(kms), max(kms)), 2),
        direction=choice(directions),
        lane=choice(lanes),
        firm=firm,
        created_by=firm.manager,
        occurrence_type=occ_type,
        form_data=get_form_data(occ_type),
        status=status,
        executed_at=executed,
        created_at=created,
        updated_at=updated,
        found_at=found,
        job=job,
    )

In [ ]:
Job.objects.filter(company=company, reportings__isnull=True).delete()

In [ ]:
def add_reporting_number(instance):
    if instance.occurrence_type:
        try:
            occurrence_kind = instance.occurrence_type.occurrence_kind
        except Exception:
            raise serializers.ValidationError("Occurrence Kind not found!")
    else:
        occurrence_kind = ""
    key_name = "RP_name_format"
    number_format = ""
    if key_name in instance.company.metadata:
        try:
            number_format = instance.company.metadata[key_name][
                occurrence_kind
            ]
        except Exception:
            if "default" in instance.company.metadata[key_name]:
                number_format = instance.company.metadata[key_name][
                    "default"
                ]
            else:
                raise serializers.ValidationError(
                    "Variáveis de nome inválidas!"
                )
    else:
        raise serializers.ValidationError("Variáveis de nome inválidas!")
    instance_type = number_format["type"]
    # Get datetime and serial arrays
    data = get_autonumber_array(instance.company.uuid, instance_type)
    # Get company prefix
    if "company_prefix" in instance.company.metadata:
        data["prefixo"] = instance.company.metadata["company_prefix"]
    else:
        data["prefixo"] = "[{}]".format(instance.company.name)
    # Make number
    try:
        number = number_format["format"].format(**data)
    except Exception as e:
        print(e)
        # Fallback
        # UHIT-RG-2018.0001
        number = "{prefixo}-{nome}-{anoCompleto}.{serialAno}".format(**data)
    instance.number = number

Reset the numbers of the reportings so that they agree to the chronological order

In [ ]:
Sequence.objects.filter(name__icontains=str(company.uuid)).update(last=0)

In [ ]:
reportings = []
for reporting in Reporting.objects.filter(company=company).order_by('found_at'):
    add_reporting_number(reporting)
    reportings.append(reporting)
bulk_update(reportings)

In [ ]:
from helpers.apps.job import total_and_executed_reporting

Archive 90% of the Job objects where all the reportings were already executed

In [ ]:
for job in Job.objects.filter(company=company):

    total, executed = total_and_executed_reporting(job)

    if total == executed and choice([False, True, True, True, True, True, True, True, True, True]):
        job.archived = True
        job.save()

## Create reportings for classe "diário" 

In [ ]:
from helpers.km_converter import get_road_coordinates

In [ ]:
diario = OccurrenceType.objects.filter(company=company, occurrence_kind="6")[0]

In [ ]:
today_date = timezone.now()
last_year = today_date.replace(year=today_date.year-1)

In [ ]:
Reporting.objects.filter(occurrence_type=diario).delete()

In [ ]:
road = Road.objects.filter(company=company).first()
point, road = get_road_coordinates(
    road.name,
    150,
    "0",
    company,
)
reportings = []
for i in tqdm(range(365)):
    day = last_year + timedelta(days=i, hours=5)
    firm = firm_admins
    
    rep = Reporting(
        company=company,
        road_name=road.name,
        road=road,
        km=100,
        end_km=100,
        point = point,
        direction=choice(directions),
        lane=choice(lanes),
        firm=firm,
        created_by=firm.manager,
        occurrence_type=diario,
        form_data=get_form_data(diario),
        status=status_identificado,
        created_at=day,
        found_at=day,
    )
    reportings.append(rep)

Reporting.objects.bulk_create(reportings)

## Measurement

Create 12 of these objects, one per month in the past 12 months. After creating the Reportings, randomly choose a few with status "Executado" and add them to the Measurement where the Reporting executed_at is within the dates of the Measurement

In [ ]:
from dateutil.relativedelta import relativedelta

def get_first_day(dt, d_years=0, d_months=0):
    y, m = dt.year + d_years, dt.month + d_months
    a, m = divmod(m-1, 12)
    first_day = date(y+a, m+1, 1)
    ret = (datetime.combine(first_day, datetime.min.time()) + timedelta(hours=5)).replace(hour=0, minute=0)
    return ret

def get_last_day(dt):
    last_day = get_first_day(dt, 0, 1) + timedelta(-1)
    ret = (datetime.combine(last_day, datetime.min.time()) + timedelta(hours=5)).replace(hour=23, minute=59)
    return(ret)

In [ ]:
Measurement.objects.filter(company=company).delete()

In [ ]:
first = Measurement.objects.create(
    company=company,
    start_date=datetime(2018, 12, 1, tzinfo=timezone.now().tzinfo),
    end_date=datetime(2018, 12, 31, tzinfo=timezone.now().tzinfo),
    approved=True,
    created_by=user,
    number = "0"
)
create_services_from_measurement(first)

for i in reversed(range(13)):
    d = timezone.now().replace(hour=0, minute=0, second=0)
    month = d - relativedelta(months=i)
    user = choice(Firm.objects.filter(company=company)).manager
    
    instance = Measurement.objects.create(
        company=company,
        start_date=get_first_day(month),
        end_date=get_last_day(month),
        approved=False,
        created_by=user,
        number = str(13 - i),
        previous_measurement= first if i == 12 else instance
    )
    
    # call function to create MeasurementServices
    create_services_from_measurement(instance)

In [ ]:
executados = Reporting.objects.filter(company=company, status=status_executado)
print(executados.count())

for item in tqdm(executados):
    try:
        measurement = Measurement.objects.filter(
            company=company,
            start_date__lte=item.executed_at,
            end_date__gt=item.executed_at
        )[0]
        # call function to update services and usages
        reportings = Reporting.objects.filter(pk=item.pk)
        create_or_update_services_and_usages(measurement, reportings)
    except Exception as e:
        print(e, item.executed_at)
        continue


## GoalAggregate

Create 12 of these objects, one per month in the past 12 months.

In [ ]:
GoalAggregate.objects.filter(company=company).delete()

In [ ]:
for i in range(12):
    d = date.today()
    month = d - relativedelta(months=i)
    
    GoalAggregate.objects.create(
        company=company,
        start_date=get_first_day(month),
        end_date=get_last_day(month),
        group_goals={
            "signaling": random.randint(20000, 70000),
            "other_services": random.randint(15000, 60000),
            "pavement": random.randint(30000, 80000),
        }
    )

## Goal

For each GoalAggregate, and for each OccurrenceType, create one Goal with a random amount

In [ ]:
goals = GoalAggregate.objects.filter(company=company)
types = OccurrenceType.objects.filter(company=company)

for goal in goals:
    for item in types:
        try:
            service = ServiceSpecs.objects.filter(occurrence_type=item)[0].service
        except:
            continue
        else:
            Goal.objects.create(
                aggregate=goal,
                occurrence_type=item,
                service=service,
                amount=random.randint(20, 50),
                internal=True
#                 internal=(item in internal_types)
            )

## ReportingFile

Create 3000 new objects

Do a google image search for "bad road" and pick a random image, associate with a random Reporting

In [ ]:
available_files = []

for form in tqdm(OccurrenceType.objects.filter(company=company)):
    normalized_name = clean_latin_string(form.name).lower().replace(' ', '_')
    file_choices = ReportingFile.upload.field.storage.listdir("images_mock/{}".format(normalized_name))[1]
    file_choices = [a for a in file_choices if ".jpg" in a]
    available_files += ["images_mock/{}/{}".format(normalized_name, a) for a in file_choices]

In [ ]:
ReportingFile.objects.filter(reporting__company=company).delete()

In [ ]:
file_objects = []

reportings = (Reporting.objects.filter(company=company)
              .exclude(occurrence_type__occurrence_kind__in=["2", "6"])
              .prefetch_related('created_by'))

for i in tqdm(range(1500)):
    reporting = choice(reportings)
    user = reporting.created_by
    normalized_path = clean_latin_string(reporting.occurrence_type.name).lower().replace(' ', '_')
    file_choices = [a for a in available_files if a.split('/')[1] == normalized_path]
    if not len(file_choices):
        continue
    file_path = choice(file_choices)
    
    rf = ReportingFile(
        reporting=reporting,
        upload=file_path,
        created_by=user,
        include_rdo=True
    )
    file_objects.append(rf)
    
for reporting in tqdm(Reporting.objects.filter(company=company, occurrence_type__occurrence_kind="2")):
    user = reporting.created_by
    normalized_path = clean_latin_string(reporting.occurrence_type.name).lower().replace(' ', '_')
    file_choices = [a for a in available_files if a.split('/')[1] == normalized_path]
    if not len(file_choices):
        continue
    file_path = choice(file_choices)
    
    rf = ReportingFile(
        reporting=reporting,
        upload=file_path,
        created_by=user,
        include_rdo=True
    )
    file_objects.append(rf)

result = ReportingFile.objects.bulk_create(file_objects, batch_size=100)
print(len(result))

# Download images (don't run)

This code will be ran only once to get images related to a certain type of service.
Once they are downloaded, the resulting images need to be uploaded to AWS S3

In [ ]:
!pip install webdriver_manager

In [ ]:
!pip install user_agent

In [ ]:
from fixtures.mock_empreiteira.download_with_selenium import do_stuff

In [ ]:
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver

In [ ]:
webdriver.Chrome(ChromeDriverManager().install())

In [ ]:
form_names = OccurrenceType.objects.filter(company=company).values_list('name', flat=True)
form_names = [a + " rodovia" for a in form_names]
do_stuff(form_names, [""], "fixtures/images/", 10)